In [ ]:
if (!require("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install

In [ ]:
BiocManager::install("biomaRt")
library(biomaRt)

In [ ]:
library(BiocManager)
BiocManager::install('GOstats')
BiocManager::install("clusterProfiler")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("enrichplot")
#BiocManager::install("Rtools")

In [1]:
library(biomaRt)
library(org.Hs.eg.db)
library(clusterProfiler)
# Leer el archivo CSV
data <- read.csv("deSeqRes_filtrado_Lactobacillus_acidophilus_KLDS1.0901.csv")
# Filtrar los genes significativos
genes_significativos <- subset(data, padj < 0.05)
# Extraer la lista de genes significativos
lista_genes <- genes_significativos$gene_name
# Verificar si los genes son números (posiblemente IDs de Entrez)
if (all(grepl("^\\d+$", lista_genes))) {
    genes_ids <- lista_genes
} else {
    # Convertir los nombres de genes a IDs de Entrez si son símbolos
    genes_ids <- bitr(lista_genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db)$ENTREZID
}
genes_ids

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: IRanges

Loading required package: S4Vectors

[1] "220"  "3576" "1958" "6559"

In [2]:

# Convertir los IDs de Entrez a nombres de genes si son IDs de Entrez
if (all(grepl("^\\d+$", genes_ids))) {
    # Crear una conexión a la base de datos de biomart
    mart <- useMart(biomart = "ensembl", dataset = "hsapiens_gene_ensembl")
    
    # Obtener los nombres de los genes correspondientes a los IDs de Entrez
    gene_info <- getBM(attributes = c("entrezgene_id", "external_gene_name"), 
                       filters = "entrezgene_id", 
                       values = genes_ids, 
                       mart = mart)
    
    # Reemplazar los IDs de Entrez por los nombres de los genes convertidos
    genes_names <- gene_info$external_gene_name
}



In [3]:
genes_names

[1] "EGR1"    "ALDH1A3" "CXCL8"   "SLC12A3"

In [4]:
# Conectarse a la base de datos UniProt usando el conjunto de datos 'hsapiens_gene_ensembl'
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

In [5]:
# Obtener la información de las proteínas correspondientes a los nombres de los genes
info_proteinas <- getBM(attributes = c("ensembl_gene_id", "external_gene_name", "uniprotswissprot", "description"),
                        filters = "external_gene_name",
                        values = genes_names,
                        mart = mart)

In [6]:
# Filtrar para obtener solo los genes que codifican proteínas
genes_codifican_proteinas <- info_proteinas[info_proteinas$uniprotswissprot != "", ]

# Mostrar los resultados
genes_codifican_proteinas

,ensembl_gene_id,external_gene_name,uniprotswissprot,description
,<chr>,<chr>,<chr>,<chr>
2,ENSG00000184254,ALDH1A3,P47895,aldehyde dehydrogenase 1 family member A3 [Source:HGNC Symbol;Acc:HGNC:409]
3,ENSG00000070915,SLC12A3,P55017,solute carrier family 12 member 3 [Source:HGNC Symbol;Acc:HGNC:10912]
6,ENSG00000169429,CXCL8,P10145,C-X-C motif chemokine ligand 8 [Source:HGNC Symbol;Acc:HGNC:6025]
7,ENSG00000120738,EGR1,P18146,early growth response 1 [Source:HGNC Symbol;Acc:HGNC:3238]


In [7]:
# Lista de genes que codifican proteínas
genes_codifican_proteinas <- unique(genes_codifican_proteinas$external_gene_name)

# Verificar la presencia de cada gen en genes_codifican_proteinas
genes_presentes <- genes_names %in% genes_codifican_proteinas

# Mostrar los genes que están en genes_codifican_proteinas
genes_encontrados <- genes_names[genes_presentes]
cat("Genes encontrados:\n")
print(genes_encontrados)

# Mostrar los genes que no están en genes_codifican_proteinas
genes_no_encontrados <- genes_names[!genes_presentes]
cat("Genes no encontrados:\n")
print(genes_no_encontrados)

Genes encontrados:


[1] "EGR1"    "ALDH1A3" "CXCL8"   "SLC12A3"
Genes no encontrados:
character(0)


In [8]:
# Obtener los nombres de las proteínas correspondientes a los genes encontrados
proteinas <- getBM(
  attributes = c("external_gene_name", "uniprotswissprot"),
  filters = "external_gene_name",
  values = genes_encontrados,
  mart = ensembl
)

# Filtrar las filas donde el campo uniprotswissprot no esté vacío
proteinas_filtradas <- proteinas[proteinas$uniprotswissprot != "", ]

# Mostrar los nombres de las proteínas
cat("Nombres de las proteínas correspondientes a los genes encontrados:\n")
print(proteinas_filtradas$uniprotswissprot)

Nombres de las proteínas correspondientes a los genes encontrados:
[1] "P47895" "P55017" "P10145" "P18146"


In [19]:
# Filtrar las filas donde el campo uniprotswissprot no esté vacío
proteinas_filtradas <- proteinas[proteinas$uniprotswissprot != "", ]

# Obtener solo la columna de nombres de proteínas
lista_proteinas <- proteinas_filtradas$uniprotswissprot


# Guardar la lista de proteínas en una variable
lista_proteinas

[1] "P47895" "P55017" "P10145" "P18146"

In [16]:
# Asegúrate de que lista_proteinas sea un vector
lista_proteinas_2 <- as.character(lista_proteinas)

# Guardar la lista de proteínas en un archivo CSV sin la columna X y sin comillas
writeLines(lista_proteinas_2, "lista_proteinas_Lactobacillus_acidophilus_KLDS1.0901.csv")

In [17]:
# Cargar el archivo CSV de nuevo en R
lista_proteinas_cargado <- read.csv("lista_proteinas_Lactobacillus_acidophilus_KLDS1.0901.csv", header = FALSE)

# Renombrar la columna si es necesario
colnames(lista_proteinas_cargado) <- "uniprotswissprot"

In [18]:
lista_proteinas_cargado

uniprotswissprot
<chr>
P47895
P55017
P10145
P18146
